<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Control-Flow" data-toc-modified-id="Control-Flow-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Control Flow</a></span><ul class="toc-item"><li><span><a href="#Basic-Conditional-Evaluation" data-toc-modified-id="Basic-Conditional-Evaluation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Basic Conditional Evaluation</a></span></li><li><span><a href="#Extended-Conditional-Evaluation" data-toc-modified-id="Extended-Conditional-Evaluation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Extended Conditional Evaluation</a></span></li><li><span><a href="#Vector-Conditional-Evaluation" data-toc-modified-id="Vector-Conditional-Evaluation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Vector Conditional Evaluation</a></span></li><li><span><a href="#if" data-toc-modified-id="if-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span><code>if</code></a></span></li><li><span><a href="#do" data-toc-modified-id="do-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span><code>do</code></a></span></li><li><span><a href="#while" data-toc-modified-id="while-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span><code>while</code></a></span></li><li><span><a href="#Return-and-Signal" data-toc-modified-id="Return-and-Signal-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Return and Signal</a></span></li><li><span><a href="#Protected-Evaluation" data-toc-modified-id="Protected-Evaluation-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Protected Evaluation</a></span></li></ul></li><li><span><a href="#Debugging" data-toc-modified-id="Debugging-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Debugging</a></span></li><li><span><a href="#Scripts" data-toc-modified-id="Scripts-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Scripts</a></span><ul class="toc-item"><li><span><a href="#Creating-and-Loading-a-Script" data-toc-modified-id="Creating-and-Loading-a-Script-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Creating and Loading a Script</a></span></li><li><span><a href="#Blocks" data-toc-modified-id="Blocks-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Blocks</a></span></li><li><span><a href="#Passing-Parameters" data-toc-modified-id="Passing-Parameters-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Passing Parameters</a></span></li></ul></li></ul></div>

## Control Flow

### Basic Conditional Evaluation

$\$[expr_{cond}; expr_{true}; expr_{false}]$

In [1]:
$[1b;42;9*6]
$[0b;42;9*6]

42


54


The brackets in any q conditional **do not create lexical scope**. This means that variables created within the body exist in the same scope as the conditional. 

In [2]:
$[1b;a:42;a:43]
a

42


42


Although evaluation of function arguments in q is eager, evaluation of the expressions in the conditional is short circuited, meaning that **only the one selected for return is evaluated**. 

In [2]:
$[1b;a:42;b:43]
a
b

42


42


[0;31mb[0m: [0;31mb[0m

In [3]:
v:0N
$[v;`isnull;`notnull]
$[null v;`isnull;`notnull]

`isnull


`isnull


**Float nulls do not work so the above is probably an accident and you should not count on it.**

### Extended Conditional Evaluation
if .. else ..: 

$\$[expr_{cond}; [expr_{true};\dots]; [expr_{false};\dots]]$

In [5]:
v:42
$[v=42; [a:6;b:7;`Everything]; [a:`Life;b:`the;c:`Universe;a,b,c]]
$[v=43; [a:6;b:7;`everything]; [a:`Life;b:`the;c:`Universe;a,b,c]]

`Everything


`Life`the`Universe


If.. elif.. else:

$\$[expr_{cond_1}; expr_{true_1}; \dots;expr_{cond_n};expr_{true_n}; expr_{false}]$

In [6]:
a:0
$[a=0;`zero; a>0;`pos; `neg]
a:42
$[a=0;`zero; a>0;`pos; `neg]
a:-42
$[a=0;`zero; a>0;`pos; `neg]

`zero


`pos


`neg


### Vector Conditional Evaluation

$?[v_b;expr_{true};expr_{false}]$

where $v_b$ is a simple boolean list and $expr_{true}$ and $expr_{false}$ are of the same type and are either atoms or vectors that conform to $v_b$. The result conforms to $v_b$ and selects from $expr_{true}$ in positions where $v_b$ is `1b` and $expr_{false}$ in positions where $v_b$ has `0b`. **All arguments of vector-conditional are fully executed. In other words, there is no short circuiting of evaluation.**

In [7]:
L:til 10
?[0<>L mod 3; L; 42]

42 1 2 42 4 5 42 7 8 42


In [8]:
t:([] c1:1.1 2.2 3.3; c2:10 20 30; c3:100 200 300)
update mix:?[c1>2.0; c3; c2] from t

c1  c2 c3  mix
--------------
1.1 10 100 10 
2.2 20 200 200
3.3 30 300 300


There are no extended forms of Vector Conditional. You can get a cascading effect by nesting vector conditional expressions.

In [9]:
update band:?[c2 within 5 15; 1; ?[c2 within 16 25; 2; 3]] from t

c1  c2 c3  band
---------------
1.1 10 100 1   
2.2 20 200 2   
3.3 30 300 3   


### `if`

$if[expr_{cond};expr_1; …;expr_n]$

The $expr_{cond}$ is evaluated and if it is non-zero the expressions $expr_1$ thru $expr_n$ are evaluated in left-to-right order.

In [13]:
a:42
b:98.6
if[a=42;x:6;y:7;b:a*b]
x
y
a
b

6


7


42


4141.2


**Well-written q code rarely needs if. One example of legitimate use is pre-checking function arguments to abort execution for bad values.**

### `do`

$do[expr_{count};expr_1; …;expr_n]$

The $expr_{count}$ must evaluate to an non-negative integer.The expressions $expr_1$ thru $expr_n$ are evaluated $expr_{count}$ times in left-to-right order.

**The best recommendation about usage of do is: Don’t!**

In [14]:
n:5
do[-1+f:r:n; r*:f-:1] / do not do this!
r

120


### `while`

$while[expr_{cond};expr_1; …;expr_n]$

The $expr_{cond}$ is evaluated and the expressions $expr_1$ thru $expr_n$ are evaluated repeatedly in left-to-right order as $expr_{cond}$ is non-zero.  The `while` statement is not a function, does not have an explicit result and does not introduce lexical scope.

**The author has never used while in actual code.**

In [15]:
f:r:n:5
while[f-:1;r*:f] / do not do this either!
r

120


### Return and Signal

There are two mechanisms for ending the execution early: one indicates successful completion and the other signals abrupt termination.

To terminate function application immediately and return a normal value, use an empty assignment – that is, `:` with the return value to its right and no variable to its left.

In [16]:
f:{0N!"Begin"; a:x; b:y; :a*b; "End"}
f[6;7]

"Begin"


42


To abort function execution immediately with an exception, use Signal, which is single-quote `'`, with an error message to its right. The error message can be provided as a symbol or string.

In [0]:
g:{0N!"Begin"; a:x; b:y; '"End"; c:b}
g[6;7]

"Begin"


[0;31mEnd[0m: [0;31mEnd[0m

Legitimate use of the `if` statement is to terminate execution with an exception.
```
{
...
if[a<50; '"Bad a"];
...
}
```

### Protected Evaluation

Q provides a try-catch capability using ternary forms of `Apply` and `Apply At`. Ternary `@` is used for unary functions and ternary `.` is used for multivalent functions. The syntax is the same for both.

$@[f_{mon};a;expr_{fail}]$

$.[f_{mul};L_{args};expr_{fail}]$

Here $f_{mon}$ is a unary function, a is single argument, $f_{mul}$ is a multivalent function, $L_{args}$ is a list of arguments, and $expr_{fail}$ is an expression or function. In both forms, the function is applied to its argument(s). Upon successful application, protected evaluation returns the result of the application. Should an exception arise, $expr_{fail}$ is applied to the resulting error string.

**You can use protected evaluation to log error messages from exceptions that would otherwise crash your program.**


**If the application of $expr_{fail}$ results in an exception, the protected call itself will fail.**


In [2]:
s:"6*7"
@[value; s; show]

42


In [3]:
s:"6*`7"
@[value; s; show]

"type"


In [4]:
prod:{x*y}
.[prod; (6;7); show]

42


In [5]:
.[prod; (6;`7); show]

"type"


## Debugging

Set a breakpoint by insert a line that you know will fail - use an undefined name. **Make sure the name you choose is not defined in local or global scope.**

In [5]:
f:{a:x*x; b:y*y; a+b}
f:{a:x*x; b:y*y; break; a+b}
f[3;4]

[0;31mbreak[0m: [0;31mbreak[0m

In [5]:
breakpoint:{break}
f:{a:x*x; b:y*y; breakpoint[]; a+b}
f[3;4]

[0;31mbreak[0m: [0;31mbreak[0m

**You will spend much more time trying to figure out why your q code is not doing what you want than the time spent doing manual debugging.**

In [5]:
zs:{`d`P`L`G`D!(system"d"),v[1 2 3],enlist last v:value x}
b:7
f:{a:6;x+a*b}
f[`100] /this is an error

[0;31mtype[0m: [0;31mtype[0m

In [6]:
show zs f

d| `.
P| ,`x
L| ,`a
G| ``b
D| "{a:6;x+a*b}"


In [6]:
zs .z.s

[0;31mnyi[0m: [0;31mnyi[0m

## Scripts

A _script_ is a q program stored in a text file with an extension of `.q` (or `.k` if you are writing k code). A script can contain any q expressions or commands. The contents of the script are parsed and evaluated sequentially from top to bottom. Global entities created during execution of the script exist in the workspace after the script is loaded and executed.



### Creating and Loading a Script

A script can be loaded at any time during a session using the `\l` command, called _load_. The load command can be executed programmatically using `system`. See Chapter 12 for more on commands.

### Blocks
Multi-line expressions are permitted in a script but they have a special form.

- The **first line must not be indented** – i.e., it begins at the left of the line with no initial whitespace.
- Any continuation lines **must** be indented, meaning that there is at least one whitespace character at the beginning of the line.
- In particular, if you put the closing brace to a function definition on its own line, it **must** be indented.
- Empty lines and comment lines (beginning with `/`) are permitted anywhere.

Table definition and function definition provide nice opportunities for splitting across multiple lines:

- A table can have line breaks after a closing square bracket `]` or after a semicolon separator `;`
- A function can have line breaks after a closing square bracket `]` or after a comma separator `,`.


```
a:42
b:0
/
this is a block of
comment text
b:42
and b will not be changed
\
a:43 / this line will be executed
\
nothing from here on will be executed
b:44
```

In [7]:
/
hey this is a command
hey
\
12

12


### Passing Parameters

Parameters are passed to a q script at q startup similarly to `argv` command line parameters in C. Specifically, the system variable `.z.x` comprises a list of strings, each containing the character representation of an argument present on the command line that invoked the script. For example, let’s modify our `trades.q` script to pass the number of records to be created as a command line parameter. Note that we parse the passed string to an integer.